In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lish-moa/test_features.csv
/kaggle/input/lish-moa/sample_submission.csv
/kaggle/input/lish-moa/train_features.csv
/kaggle/input/lish-moa/train_targets_scored.csv
/kaggle/input/lish-moa/train_targets_nonscored.csv


## Read Input Files

In [2]:
train_data = pd.read_csv(os.path.join('/kaggle/input/lish-moa', 'train_features.csv'))
test_data = pd.read_csv(os.path.join('/kaggle/input/lish-moa' , 'test_features.csv'))

train_target = pd.read_csv(os.path.join('/kaggle/input/lish-moa','train_targets_scored.csv'))

## Print Shape of Data

In [3]:
print("train_data.shap: {}  , train_target.shap: {}  , test_data.shap: {} ".format(train_data.shape , train_target.shape , test_data.shape))

train_data.shap: (23814, 876)  , train_target.shap: (23814, 207)  , test_data.shap: (3982, 876) 


In [4]:
NGTypeFeature = sum(train_data.columns.to_series().str.contains('g-') == True )
NCTypeFeature = sum(train_data.columns.to_series().str.contains('c-') == True )

print("NGTypeFeature = {} \nNCTypeFeature = {}".format(NGTypeFeature , NCTypeFeature))

NGTypeFeature = 772 
NCTypeFeature = 100


## Define a Neural Network To Train

In [112]:
import torch.nn.functional as F
import torch.nn as nn

class MultiLabelClassifer(nn.Module):
    #TO-DO: Documentation
    def __init__(self , input_feature , hidden_dim , output_dim):
        super(MultiLabelClassifer , self).__init__()
        
        #nn.utils.weight_norm
        self.fc1 = (nn.Linear(in_features = input_feature , out_features = 2000))
        #TO-DO: As close features are related here, convoluation layer will help here
        #TO-DO: Later add and play with Convolutatin layer, to see accurracy difference
        self.fc2 =  (nn.Linear( in_features = 2000 , out_features  = 4000))
        self.fc3 = (nn.Linear( in_features = 4000 , out_features  = 1000))
        self.fc4 =  (nn.Linear(in_features = 1000 , out_features = 400))
        self.fc5 = (nn.Linear(in_features = 400 , out_features = output_dim))
        self.drop = nn.Dropout(0.4) # dropout with 30% prob
        
        self.out = (nn.Linear(output_dim,output_dim)) #Above using weight normalization for final layer
        #Query: Why use weight normalization. Checkout this awesome stackoverlfow post:
        #link: https://stats.stackexchange.com/questions/352036/what-should-i-do-when-my-neural-network-doesnt-learn
        
        self.sig = nn.Sigmoid()
        #self.F_Sig = F.sigmoid() #signout output range from [0,1]
        self.Log_Sig = nn.LogSigmoid() #log of range 0-1 is whole number?
        self.soft = nn.Softmax()  #sum of all output is '1'. So its used only in multi-class problem.
        #self.linear3 = nn.utils.weight_norm(nn.Linear(hidden_size,output_size))
        #TO-DO: Define output layer so that it be as per Question requirement
    def forward(self, x):
        
        # (1) input layer
        
        t = x
        
        
        # (2) Hidden Linear Layer
        
        t = self.fc1(t)
        t = F.relu(t)
        #3
        
        t = self.fc2(t)
        t = F.relu(t)
        #4
        
        t= self.fc3(t)
        t = F.relu(t)
        t = self.drop(t)
        
        #5
        
        t = self.fc4(t)
        t = F.relu(t)
        #6
        
        t = self.drop(t)
        
        t = self.fc5(t)
        t = F.relu(t)
        #t = self.F_Sig(t)
        #t = F.sigmoid(t)
        #t = self.Log_Sig(t)
        
        # (7) Output Layer
        
        t = self.out(t) #This working okay. but why? 
        
        #t = self.sig(t)  #Using Sigmmoid ALSO as last layer is causing training loss to not decrease
        #t=self.soft(t) #==> Using softmax as last layer is causing training loss to not decrease
        
        return t
        

In [113]:
train_rf = train_data.copy()
train_data.head()

test_rf = test_data.copy()
train_rf.head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


## One hot coding of input feature and Normalization

In [114]:
import torch.utils.data
import torch
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,Normalizer

ignore_columns = ['sig_id','cp_type']

train_columns = [x for x in train_data.columns if x not in ignore_columns]
print(len(train_columns))
train = train_rf[train_columns]
idx = train.index
col = train.columns
print("size of train is {}",train.shape)
train.head()





874
size of train is {} (23814, 874)


,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,-0.0326,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,0.3372,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,0.2155,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,0.1792,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,-0.1498,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


In [115]:
test = test_rf[train_columns]
target = train_target.iloc[:,1:].values
#https://stackoverflow.com/questions/54160370/how-to-use-sklearn-column-transformer/54160620
transform = ColumnTransformer([('o',OneHotEncoder(),[0,1]),('s',Normalizer(),list(range(3,train.shape[1])))])
print("size before transform train.shape:{} test.shape{}".format(train.shape,test.shape))
train = transform.fit_transform(train)
test = transform.transform(test)

print("size AFTER transform train.shape:{} test.shape{}".format(train.shape,test.shape))

size before transform train.shape:(23814, 874) test.shape(3982, 874)
size AFTER transform train.shape:(23814, 876) test.shape(3982, 876)


In [116]:
#train_rf['cp_dose'] = train_data['cp_dose'].replace({"D1":1 , "D2":2})
#train_rf['cp_type'] = train_rf['cp_type'].replace({"trt_cp":1 , "ctl_vehicle":2})

#test_rf['cp_dose'] = test_data['cp_dose'].replace({"D1":1 , "D2":2})
#test_rf['cp_type'] = test_data['cp_type'].replace({"trt_cp":1 , "ctl_vehicle":2})
print(type(train))
print(len(col))
print(len(idx))
print(train.shape)
print(type(col))

#col = pd.concat(['p1', 'p2'] ,col)
train_df = pd.DataFrame(data = train,index = idx ) #Check if encoding is successful or not


<class 'numpy.ndarray'>
874
23814
(23814, 876)
<class 'pandas.core.indexes.base.Index'>


In [117]:
train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,866,867,868,869,870,871,872,873,874,875
0,1.0,0.0,0.0,1.0,0.0,0.025030,-0.011126,-0.027862,-0.008725,-0.045420,...,0.012845,0.011597,0.036246,0.024788,-0.008581,0.029550,-0.017867,0.009600,0.017059,0.018742
1,0.0,0.0,1.0,1.0,0.0,0.018548,0.013574,0.002741,0.046245,0.023631,...,-0.019356,0.034232,0.021366,0.001044,0.013420,0.022233,0.006907,0.005632,0.027579,0.033451
2,0.0,1.0,0.0,1.0,0.0,0.020432,0.054585,-0.002684,-0.001135,0.043520,...,-0.025466,-0.022118,0.021437,0.000783,-0.046506,-0.011149,-0.022540,-0.007682,-0.049456,0.024345
3,0.0,1.0,0.0,1.0,0.0,-0.006312,-0.006730,0.013400,0.102930,-0.020513,...,-0.053188,-0.016321,-0.142663,-0.034918,-0.021873,-0.032638,-0.041076,-0.022259,-0.009822,-0.020662
4,0.0,0.0,1.0,0.0,1.0,-0.014177,0.034302,0.024468,0.050145,-0.029153,...,0.000149,0.000170,0.023587,0.037803,0.019531,-0.010719,0.003869,0.010202,-0.013388,0.025196


## Get Data Tensor

In [118]:
print(type(train))
train_data_tensor =  torch.from_numpy(train.astype(np.float32))
train_target_tensor = torch.from_numpy(target.astype(np.float32))

<class 'numpy.ndarray'>


In [119]:
print("max_val {}, min_val {}",train_data_tensor.max() , train_data_tensor.min() )
#Value range b/w -1 to 1 implies that it has been normalized

max_val {}, min_val {} tensor(1.) tensor(-0.4424)


In [120]:
print(train_target_tensor)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [121]:
print(sum(sum(train_target_tensor)))

tensor(16844.)


In [122]:
print("type of train_data_tensor: {}",type(train_data_tensor) )
train_set = torch.utils.data.TensorDataset(train_data_tensor , train_target_tensor)
print(train_set)

type of train_data_tensor: {} <class 'torch.Tensor'>


## Get dataloader

In [123]:
batch_size = 120
train_loader = torch.utils.data.DataLoader(train_set , batch_size = batch_size, shuffle=True)
print(train_loader)

## Now take Instance of Network

In [124]:
model = MultiLabelClassifer(876 , 1024  , 206 )
#(self , input_feature , hidden_dim , output_dim):
#model = ModelTWO(876 , 206 , 1024)

# Analyser Sample Formed Data

In [125]:
batch = next(iter(train_loader))
feature, label = batch

In [126]:
print(batch[0].shape)
print(batch[0][0].shape) #THis is one extracted Feature out of 100

torch.Size([120, 876])
torch.Size([876])


In [127]:
print(feature)

tensor([[ 1.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -6.1060e-02,
          7.5089e-02, -1.1622e-02],
        [ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.3246e-02,
          3.0805e-04,  7.0302e-03],
        [ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  5.9820e-02,
         -4.4562e-03,  1.3422e-02],
        ...,
        [ 1.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.1282e-02,
         -2.9852e-02, -1.1367e-02],
        [ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ..., -5.6313e-02,
         -3.1234e-02, -5.0670e-02],
        [ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ..., -1.6008e-02,
         -4.9218e-02,  6.4635e-04]])


In [128]:
print(sum(label[0]))
print(label[0])   #label for 1st feature data

tensor(1.)
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])


# Verify if the weight are being updated.
## Unit test for weight updating or not.

In [129]:
print(model)

MultiLabelClassifer(
  (fc1): Linear(in_features=876, out_features=2000, bias=True)
  (fc2): Linear(in_features=2000, out_features=4000, bias=True)
  (fc3): Linear(in_features=4000, out_features=1000, bias=True)
  (fc4): Linear(in_features=1000, out_features=400, bias=True)
  (fc5): Linear(in_features=400, out_features=206, bias=True)
  (drop): Dropout(p=0.4, inplace=False)
  (out): Linear(in_features=206, out_features=206, bias=True)
  (sig): Sigmoid()
  (Log_Sig): LogSigmoid()
  (soft): Softmax(dim=None)
)


In [130]:
# Print model parameters
for param in model.parameters():
    print(param.shape)

torch.Size([2000, 876])
torch.Size([2000])
torch.Size([4000, 2000])
torch.Size([4000])
torch.Size([1000, 4000])
torch.Size([1000])
torch.Size([400, 1000])
torch.Size([400])
torch.Size([206, 400])
torch.Size([206])
torch.Size([206, 206])
torch.Size([206])


In [131]:
#Print Shape
for name, param in model.named_parameters():
    print(name, '\t\t', param.shape)


fc1.weight 		 torch.Size([2000, 876])
fc1.bias 		 torch.Size([2000])
fc2.weight 		 torch.Size([4000, 2000])
fc2.bias 		 torch.Size([4000])
fc3.weight 		 torch.Size([1000, 4000])
fc3.bias 		 torch.Size([1000])
fc4.weight 		 torch.Size([400, 1000])
fc4.bias 		 torch.Size([400])
fc5.weight 		 torch.Size([206, 400])
fc5.bias 		 torch.Size([206])
out.weight 		 torch.Size([206, 206])
out.bias 		 torch.Size([206])


In [132]:
print(model.fc1)
#Weight Before Model train
print(model.fc1.weight)

Linear(in_features=876, out_features=2000, bias=True)
Parameter containing:
tensor([[ 0.0081,  0.0327, -0.0197,  ..., -0.0287, -0.0120, -0.0078],
        [-0.0303,  0.0296, -0.0299,  ...,  0.0178,  0.0132,  0.0007],
        [-0.0118, -0.0266, -0.0329,  ..., -0.0321,  0.0110, -0.0142],
        ...,
        [-0.0262,  0.0044,  0.0135,  ...,  0.0184, -0.0328,  0.0237],
        [ 0.0212, -0.0095, -0.0241,  ...,  0.0204,  0.0080,  0.0076],
        [-0.0013,  0.0298, -0.0049,  ...,  0.0338, -0.0274, -0.0183]],
       requires_grad=True)


## Train model for one batch

In [133]:
import torch.optim as optim
batch = next(iter(train_loader))
feature, label = batch
loss_fn = nn.BCEWithLogitsLoss()

criterion = torch.nn.BCELoss()
optimizer = optim.Adam(model.parameters() , .01)

In [134]:
preds = model(feature)
loss = loss_fn(preds , label)
loss.backward()
optimizer.step() #update weights

In [135]:
#Print Weight After model train
print(model.fc1.weight)

Parameter containing:
tensor([[-0.0017,  0.0424, -0.0292,  ..., -0.0198, -0.0154, -0.0136],
        [-0.0206,  0.0395, -0.0299,  ...,  0.0253,  0.0066,  0.0075],
        [-0.0018, -0.0167, -0.0239,  ..., -0.0292,  0.0185, -0.0151],
        ...,
        [-0.0362, -0.0055,  0.0035,  ...,  0.0258, -0.0406,  0.0321],
        [ 0.0112, -0.0194, -0.0341,  ...,  0.0118,  0.0015,  0.0164],
        [-0.0112,  0.0397,  0.0050,  ...,  0.0258, -0.0356, -0.0187]],
       requires_grad=True)


In [136]:
#Print prediction
prd = model(feature)
print((label[0])) #lable for one data set

print(prd[0]) #NOTE: if Sigmoid is not used as final layer activation funciton ==> these prediction will not be range [0,1]. Which is wrong, as output lable in in range [0,1]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])
tensor([-15.9391, -25.4950, -22.7615, -26.5382, -22.0151, -42.7090, -29.3274,
 

# Now train the model

In [139]:

import torch.optim as optim
learning_rate = [ .01]

for lr in learning_rate:
    optimizer = optim.Adam(model.parameters() , lr = lr)
    #criterion = torch.nn.BCELoss()
    criterion = nn.BCEWithLogitsLoss()
    #Difference b/w above two loss => https://discuss.pytorch.org/t/bceloss-vs-bcewithlogitsloss/33586

    for epoch in range(6):
        model.train()
        total_loss = 0
        total_correct = 0

        for batch in train_loader:
            features , labels  = batch
            optimizer.zero_grad()

            preds = model(features)

            #print(sum(preds>1))
            loss = criterion((preds) , labels)


            loss.backward()
            optimizer.step() #update weights

            total_loss += loss.data.item()
            #print("total_loss: {}".format(total_loss))
            #total_correct  += get_num_correct(preds,labels)
        print(" epoch: {} , learning_rate:{} , total_loss: {}".format(epoch, lr, total_loss))

 epoch: 0 , learning_rate:0.01 , total_loss: 5.344890164211392
 epoch: 1 , learning_rate:0.01 , total_loss: 3.781572419218719
 epoch: 2 , learning_rate:0.01 , total_loss: 3.5999912759289145
 epoch: 3 , learning_rate:0.01 , total_loss: 3.5812499364838004
 epoch: 4 , learning_rate:0.01 , total_loss: 3.551726679317653
 epoch: 5 , learning_rate:0.01 , total_loss: 3.580019478686154


In [140]:
print("DONe")

DONe


In [142]:
print("20 epochssddsfdsd")



20 epochssdfdsd
